In [2]:
import pandas as pd
import numpy as np
import os

In [2]:
path = "F:\\weather\\data txt\\txt\\" #文件夹目录
files= os.listdir(path)

In [3]:
len(files)

4006

In [14]:
data_m=pd.read_table(path+files[0],delim_whitespace=True)#多个空值作分隔符

In [15]:
data_m=data_m[data_m['Station_Id_C']==58135]

In [16]:
data_m.head()

Empty DataFrame
Columns: [Station_Id_C, Year, Mon, Day, Hour, WIN_S_Max, RHU, WIN_D_S_Max, PRS_Min, TEM_Min, PRS_Max, RHU_Min, PRS_Sea, VAP, PRE_1h, WIN_S_Inst_Max, PRS, WIN_D_INST_Max, TEM_Max, WIN_D_Avg_10mi, TEM, WIN_S_Avg_10mi]
Index: []

[0 rows x 22 columns]

In [20]:
i=1
while i<=len(files)-1:
    dataset=pd.read_table(path+files[i],delim_whitespace=True)
    data_set=dataset[dataset['Station_Id_C']==58135]
    data_m=pd.concat([data_m,data_set])
    i+=1
else:print('merge complete!')

IndexError: list index out of range

In [13]:
data_m.to_csv('F:\\weather\\xw-sh.csv',index=False,sep=',')

# 气象数据分省份整理并画图

## 北京

In [3]:
path = "F:\\weather\\data txt\\txt\\" #气象文件夹目录
files= os.listdir(path)

In [4]:
station=pd.read_excel('F:\\weather\\station.xlsx',usecols=[0,1]) #省份与站号的对应表
station.rename(columns={'区站号':'Station_Id_C'},inplace=True)

In [5]:
station.head()

省份  Station_Id_C
0  安徽         58015
1  安徽         58016
2  安徽         58102
3  安徽         58107
4  安徽         58108

### 提取所有北京站点数据

In [6]:
sta_bj=station[station['省份']=='北京']

In [7]:
sta_bjs=sta_bj['Station_Id_C'].values.tolist() #dataframe转列表，方便后续使用lambda函数

In [8]:
data_m=pd.read_table(path+files[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件

In [9]:
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_bjs else 0 )

In [10]:
data_n=data_m[data_m['Station_Id_C']>0]

In [11]:
i=1
while i<=len(files)-1:
    data_set=pd.read_table(path+files[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_bjs else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_n=pd.concat([data_n,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


In [74]:
data_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193709 entries, 0 to 2879
Data columns (total 37 columns):
CLO_COV_LM        67340 non-null float64
CLO_Cov           67340 non-null float64
CLO_Cov_Low       67340 non-null float64
Day               193709 non-null float64
Day_Data          0 non-null object
Hour              193709 non-null float64
Hour_Data         0 non-null object
Mon               193709 non-null float64
Mon_Data          0 non-null object
PRE_1h            193709 non-null float64
PRS               193709 non-null float64
PRS_Max           193709 non-null float64
PRS_Min           193709 non-null float64
PRS_Sea           193709 non-null float64
RHU               193709 non-null object
RHU_Min           193709 non-null float64
Station_Id_C      193709 non-null object
TEM               193709 non-null float64
TEM_Max           193709 non-null float64
TEM_Min           193709 non-null float64
VAP               193709 non-null float64
VIS               78527 non-null

合并后总计有193,709行数据，下一步进行数据整理

### 数据整理

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [12]:
data_bj=data_n[:]

In [13]:
data_bj=data_n[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [14]:
data_bj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193709 entries, 0 to 2879
Data columns (total 9 columns):
Station_Id_C    193709 non-null object
Year            193709 non-null float64
Mon             193709 non-null float64
Day             193709 non-null float64
Hour            193709 non-null float64
RHU             193709 non-null object
TEM             193709 non-null float64
TEM_Max         193709 non-null float64
TEM_Min         193709 non-null float64
dtypes: float64(7), object(2)
memory usage: 14.8+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [15]:
data_bj_drop=data_bj.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [16]:
data_bj_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173825 entries, 0 to 2879
Data columns (total 9 columns):
Station_Id_C    173825 non-null object
Year            173825 non-null float64
Mon             173825 non-null float64
Day             173825 non-null float64
Hour            173825 non-null float64
RHU             173825 non-null object
TEM             173825 non-null float64
TEM_Max         173825 non-null float64
TEM_Min         173825 non-null float64
dtypes: float64(7), object(2)
memory usage: 13.3+ MB


去重前193,709条数据，去重后173,825条数据。

In [17]:
data_beijing=data_bj_drop[:]
data_beijing['Station_Id_C']=data_beijing['Station_Id_C'].apply(lambda s :int(s))

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [18]:
sta_bj.count()

省份              18
Station_Id_C    18
dtype: int64

In [19]:
data_beijing_f=data_beijing[:]

In [20]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_beijing_f['省份']='beijing'
data_beijing_f['Area']='beijing'

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
data_beijing_f.count()

Station_Id_C    173825
Year            173825
Mon             173825
Day             173825
Hour            173825
RHU             173825
TEM             173825
TEM_Max         173825
TEM_Min         173825
省份              173825
Area            173825
dtype: int64

北京地区站点共有18个，筛选后的数据173,825条。

In [22]:
data_beijing_f.to_csv('F:\\weather\\beijing.csv',index=False,encoding='utf_8_sig',sep=',')

__核对温度是否符合：温度最大、最小、平均值之间逻辑关系：最小≤平均≤最大__

In [23]:
data_bj_valid=data_bj_drop[:]

In [24]:
data_bj_valid['valid1']=data_bj_drop['TEM_Max']-data_bj_drop['TEM']
data_bj_valid['valid2']=data_bj_drop['TEM']-data_bj_drop['TEM_Min']

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
data_valid=data_bj_valid[(data_bj_valid['valid1']>=0) | (data_bj_valid['valid2']>=0)]

In [26]:
data_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173825 entries, 0 to 2879
Data columns (total 11 columns):
Station_Id_C    173825 non-null object
Year            173825 non-null float64
Mon             173825 non-null float64
Day             173825 non-null float64
Hour            173825 non-null float64
RHU             173825 non-null object
TEM             173825 non-null float64
TEM_Max         173825 non-null float64
TEM_Min         173825 non-null float64
valid1          173825 non-null float64
valid2          173825 non-null float64
dtypes: float64(9), object(2)
memory usage: 15.9+ MB


可见所有温度数据符合逻辑。

__统计“测定站-年-月-日”，当天数据量≥16条的保留（总共24条），其余剔除__

In [27]:
data_bjh=data_bj_drop[:]

In [28]:
data_bjh['Station_Id_C']=list(map(lambda s: str(s),data_bjh['Station_Id_C']))
data_bjh['Year']=list(map(lambda s: str(s),data_bjh['Year']))
data_bjh['Mon']=list(map(lambda s: str(s),data_bjh['Mon']))
data_bjh['Day']=list(map(lambda s: str(s),data_bjh['Day']))

In [29]:
data_bjh['SYMD']=data_bjh['Station_Id_C']+'-'+data_bjh['Year']+'-'+data_bjh['Mon']+'-'+data_bjh['Day']

In [30]:
valid=data_bjh['SYMD'].value_counts()
valid1=valid.to_frame('SYMD_count')

In [31]:
data_bjh1=pd.merge(data_bjh,valid1,left_on='SYMD',right_index=True)

In [32]:
data_bjhf=data_bjh1[data_bjh1['SYMD_count']>=16]

In [33]:
data_bjhf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165293 entries, 0 to 2879
Data columns (total 11 columns):
Station_Id_C    165293 non-null object
Year            165293 non-null object
Mon             165293 non-null object
Day             165293 non-null object
Hour            165293 non-null float64
RHU             165293 non-null object
TEM             165293 non-null float64
TEM_Max         165293 non-null float64
TEM_Min         165293 non-null float64
SYMD            165293 non-null object
SYMD_count      165293 non-null int64
dtypes: float64(4), int64(1), object(6)
memory usage: 15.1+ MB


保留了足够重复数据165,293行

__统计“测定站-年-月”，当月数据量≥15天以上保留（总共28-31条），其余剔除__

In [34]:
# data_bjd=data_bjhf[:]

# data_bjd['SYM']=data_bjd['Station_Id_C']+'-'+data_bjd['Year']+'-'+data_bjd['Mon']

# data_bjd1=data_bjd.drop_duplicates('SYMD',keep='first')

# valid=data_bjd1['SYM'].value_counts()
# valid1=valid.to_frame('SYM_count')

# data_bjd2=pd.merge(data_bjd,valid1,left_on='SYM',right_index=True,how='left')

# data_bjdf=data_bjd2[data_bjd2['SYM_count']>=15]

# data_bjdf.info()

保留了足够重复数据154,408行。

__计算北京市温湿度指数（THI），取平均温度计算。计算公式：THI=0.81*温度+(0.99*温度-14.3)*相对湿度/100+46.3__

In [37]:
data_bj_final=data_bjhf[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [38]:
data_bj_final['Year']=list(map(lambda s: float(s),data_bj_final['Year']))
data_bj_final['Mon']=list(map(lambda s: float(s),data_bj_final['Mon']))
data_bj_final['Day']=list(map(lambda s: float(s),data_bj_final['Day']))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [39]:
data_bj_final['THI']=0.81*data_bj_final['TEM']+(0.99*data_bj_final['TEM']-14.3)*(data_bj_final['RHU']/100)+46.3

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
data_bj_final.head()

Station_Id_C    Year  Mon  Day  Hour RHU  TEM  TEM_Max  TEM_Min      THI
0        54398  2016.0  1.0  4.0   0.0  43 -2.9     -2.4     -3.1  36.5675
1        54398  2016.0  1.0  4.0   1.0  43 -1.7     -1.7     -3.1  38.0503
2        54398  2016.0  1.0  4.0   2.0  36 -0.4     -0.4     -1.7  40.6854
3        54398  2016.0  1.0  4.0   3.0  29  0.8      1.0     -0.4  43.0307
4        54398  2016.0  1.0  4.0   4.0  27  1.1      1.3      0.5   43.624

In [41]:
data_bj_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165293 entries, 0 to 2879
Data columns (total 10 columns):
Station_Id_C    165293 non-null object
Year            165293 non-null float64
Mon             165293 non-null float64
Day             165293 non-null float64
Hour            165293 non-null float64
RHU             165293 non-null object
TEM             165293 non-null float64
TEM_Max         165293 non-null float64
TEM_Min         165293 non-null float64
THI             165293 non-null object
dtypes: float64(7), object(3)
memory usage: 13.9+ MB


In [42]:
data_bj_final.to_csv("F:\\weather\\BJ-plot.csv",index=False,sep=',')

### 画图

__堆叠柱状图反映365天中THI的变化__

In [243]:
data_bj_2017=data_bj_final[data_bj_final['Year']==2017]

In [244]:
data_bj_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86430 entries, 0 to 2727
Data columns (total 11 columns):
Station_Id_C     86430 non-null object
Year             86430 non-null float64
Mon              86430 non-null float64
Day              86430 non-null float64
Hour             86430 non-null float64
RHU              86430 non-null object
TEM              86430 non-null float64
TEM_Max          86430 non-null float64
TEM_Min          86430 non-null float64
WIN_D_Avg_2mi    35423 non-null float64
THI              86430 non-null object
dtypes: float64(8), object(3)
memory usage: 7.9+ MB


In [245]:
data_bj_2017['THI']=list(map(lambda s: float(s),data_bj_2017['THI']))
data_bj_2017['Mon']=list(map(lambda s: int(s),data_bj_2017['Mon']))
data_bj_2017['Day']=list(map(lambda s: int(s),data_bj_2017['Day']))
data_bj_2017['Year']=list(map(lambda s: int(s),data_bj_2017['Year']))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [246]:
data_bj_y=data_bj_2017[data_bj_2017['THI']<=100] #删除THI异常值

In [247]:
data_bj_y['Mon']=list(map(lambda s: str(s),data_bj_y['Mon']))
data_bj_y['Day']=list(map(lambda s: str(s),data_bj_y['Day']))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [248]:
data_bj_y['M-D']=data_bj_y['Mon']+'-'+data_bj_y['Day']

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [249]:
mean_THI=data_bj_y['THI'].groupby([data_bj_y['M-D']]).mean()

In [250]:
m_THI=mean_THI.to_frame('mean_THI')

In [251]:
data_bj_f=pd.merge(data_bj_y,m_THI,left_on='M-D',right_index=True)

In [252]:
data_bj_y['Mon']=list(map(lambda s: int(s),data_bj_y['Mon']))
data_bj_y['Day']=list(map(lambda s: int(s),data_bj_y['Day']))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [253]:
data_bj_f1=data_bj_f.drop_duplicates(['Year','Mon','Day','mean_THI'],keep='first')[['Mon','Day','mean_THI']].sort_values(by=['Mon','Day'])

In [254]:
data_bj_f1.to_csv("F:\\weather\\plot2017-BJ.csv",index=False,sep=',')

## 其他地区气象数据整理

中国地区可以分为以下：华东，华南，华北，华中，西南，西北，东北。各自对应许多省份，接下来按照以下几个区域对相应的气象数据进行整理。

In [6]:
area=pd.read_excel('F:\\weather\\station.xlsx',sheet_name=2,usecols=[0,1]) #省份与地理分区对应表
area.rename(columns={'自然地理分区':'area','省市自治区':'province'},inplace=True)

In [7]:
hd=['上海','江苏','浙江','安徽','江西','山东','福建']
hb=['北京','天津','山西','河北','内蒙古']
hz=['河南','湖北','湖南']
hn=['广东','广西','海南']
xn=['四川','贵州','云南','重庆','西藏']
xb=['陕西','甘肃','青海','宁夏','新疆']
db=['黑龙江','吉林','辽宁']

### 华东地区

#### 数据筛选合并

__2016年__

In [8]:
sta_hd=station[station['省份']==hd[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(hd):
    sta_hd1=station[station['省份']==hd[i]]['Station_Id_C'].values.tolist()
    sta_hd.extend(sta_hd1)
    i+=1    #提取所有华东省份的站点ID

In [10]:
path_2016='F:\\weather\\data txt\\2016txt\\'

In [11]:
file_2016=os.listdir('F:\\weather\\data txt\\2016txt')

In [17]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hd else 0 )
data_hd=data_m[data_m['Station_Id_C']>0]

In [19]:
i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hd else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hd=pd.concat([data_hd,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [12]:
path_2017='F:\\weather\\data txt\\2017txt\\'

file_2017=os.listdir('F:\\weather\\data txt\\2017txt')

In [52]:

data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hd else 0 )
data_hd2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hd else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hd2017=pd.concat([data_hd2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [28]:
data_huad=data_hd[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [29]:
data_huad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4655373 entries, 0 to 17437
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 355.2+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [30]:
data_huad_drop=data_huad.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [31]:
data_huad_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3832313 entries, 0 to 17437
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 292.4+ MB


去重前4,655,373条数据，去重后3,832,313条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [75]:
data_huad2017=data_hd2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [76]:
data_huad2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4121009 entries, 0 to 13384
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 314.4+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [77]:
data_huad2017_drop=data_huad2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [78]:
data_huad2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3590899 entries, 0 to 13384
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 274.0+ MB


去重前4,121,009条数据，去重后3,590,899条数据。

__两年数据合并__

In [80]:
data_huadong=pd.concat([data_huad_drop,data_huad2017_drop])

In [97]:
data_huadong['Station_Id_C']=data_huadong['Station_Id_C'].apply(lambda s:int(s))
data_huadong.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7423212 entries, 0 to 13384
Data columns (total 9 columns):
Station_Id_C    int64
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), int64(1), object(1)
memory usage: 726.3+ MB


__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [110]:
sta_hd_province=station[station['省份']==hd[0]]

i=1
while i < len(hd):
    sta_hd_province1=station[station['省份']==hd[i]]
    sta_hd_province=pd.concat([sta_hd_province,sta_hd_province1])
    i+=1    #提取所有华东省份的站点ID

In [111]:
sta_hd_province.count()

省份              477
Station_Id_C    477
dtype: int64

In [148]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_huadong_f=pd.merge(data_huadong,sta_hd_province,on='Station_Id_C',how='outer')
data_huadong_f['Area']='huadong'

In [149]:
data_huadong_f.count()

Station_Id_C    7423212
Year            7423212
Mon             7423212
Day             7423212
Hour            7423212
RHU             7423212
TEM             7423212
TEM_Max         7423212
TEM_Min         7423212
省份              7423212
Area            7423212
dtype: int64

华东地区站点共有477个，筛选后的数据7,423,212条。

In [184]:
data_huadong_f.to_csv('F:\\weather\\huadong.csv',index=False,encoding='utf_8_sig',sep=',')

### 华南地区

#### 数据筛选

In [53]:
sta_hn=station[station['省份']==hn[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(hn):
    sta_hn1=station[station['省份']==hn[i]]['Station_Id_C'].values.tolist()
    sta_hn.extend(sta_hn1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [55]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hn else 0 )
data_hn2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hn else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hn2016=pd.concat([data_hn2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [56]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hn else 0 )
data_hn2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hn else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hn2017=pd.concat([data_hn2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [114]:
data_huan2016=data_hn2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [115]:
data_huan2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1630580 entries, 0 to 2559
Data columns (total 9 columns):
Station_Id_C    1630580 non-null object
Year            1630580 non-null object
Mon             1630580 non-null object
Day             1630580 non-null object
Hour            1630580 non-null object
RHU             1630580 non-null object
TEM             1630580 non-null float64
TEM_Max         1630580 non-null float64
TEM_Min         1630580 non-null float64
dtypes: float64(3), object(6)
memory usage: 124.4+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [116]:
data_huan2016_drop=data_huan2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [117]:
data_huan2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1375748 entries, 0 to 2559
Data columns (total 9 columns):
Station_Id_C    1375748 non-null object
Year            1375748 non-null object
Mon             1375748 non-null object
Day             1375748 non-null object
Hour            1375748 non-null object
RHU             1375748 non-null object
TEM             1375748 non-null float64
TEM_Max         1375748 non-null float64
TEM_Min         1375748 non-null float64
dtypes: float64(3), object(6)
memory usage: 105.0+ MB


去重前1,630,580条数据，去重后1,375,748条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [118]:
data_huan2017=data_hn2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [119]:
data_huan2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789543 entries, 0 to 2527
Data columns (total 9 columns):
Station_Id_C    789543 non-null object
Year            789543 non-null float64
Mon             789543 non-null float64
Day             789543 non-null float64
Hour            789543 non-null float64
RHU             789543 non-null object
TEM             789543 non-null float64
TEM_Max         789543 non-null float64
TEM_Min         789543 non-null float64
dtypes: float64(7), object(2)
memory usage: 60.2+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [120]:
data_huan2017_drop=data_huan2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [121]:
data_huan2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 686675 entries, 0 to 2527
Data columns (total 9 columns):
Station_Id_C    686675 non-null object
Year            686675 non-null float64
Mon             686675 non-null float64
Day             686675 non-null float64
Hour            686675 non-null float64
RHU             686675 non-null object
TEM             686675 non-null float64
TEM_Max         686675 non-null float64
TEM_Min         686675 non-null float64
dtypes: float64(7), object(2)
memory usage: 52.4+ MB


去重前789,543条数据，去重后686,675条数据。

__两年数据合并__

In [128]:
data_huanan=pd.concat([data_huan2016_drop,data_huan2017_drop])

In [130]:
data_huanan['Station_Id_C']=data_huanan['Station_Id_C'].apply(lambda s:int(s))
data_huanan.count()

Station_Id_C    2062423
Year            2062423
Mon             2062423
Day             2062423
Hour            2062423
RHU             2062423
TEM             2062423
TEM_Max         2062423
TEM_Min         2062423
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [131]:
sta_hn_province=station[station['省份']==hn[0]]

i=1
while i < len(hn):
    sta_hn_province1=station[station['省份']==hn[i]]
    sta_hn_province=pd.concat([sta_hn_province,sta_hn_province1])
    i+=1    #提取所有华东省份的站点ID

In [132]:
sta_hn_province.count()

省份              177
Station_Id_C    177
dtype: int64

In [133]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_huanan_f=pd.merge(data_huanan,sta_hn_province,on='Station_Id_C',how='outer')
data_huanan_f['Area']='huanan'

In [134]:
data_huanan_f.count()

Station_Id_C    2062423
Year            2062423
Mon             2062423
Day             2062423
Hour            2062423
RHU             2062423
TEM             2062423
TEM_Max         2062423
TEM_Min         2062423
省份              2062423
Area            2062423
dtype: int64

华南地区站点共有177个，筛选后的数据2,062,423条。

In [185]:
data_huanan_f.to_csv('F:\\weather\\huanan.csv',index=False,encoding='utf_8_sig',sep=',')

### 华北地区

#### 数据筛选

In [58]:
sta_hb=station[station['省份']==hb[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(hb):
    sta_hb1=station[station['省份']==hb[i]]['Station_Id_C'].values.tolist()
    sta_hb.extend(sta_hb1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [59]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hb else 0 )
data_hb2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hb else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hb2016=pd.concat([data_hb2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [60]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hb else 0 )
data_hb2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hb else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hb2017=pd.concat([data_hb2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [135]:
data_huab2016=data_hb2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [136]:
data_huab2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3764888 entries, 0 to 15999
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 287.2+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [137]:
data_huab2016_drop=data_huab2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [138]:
data_huab2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2976570 entries, 0 to 15999
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 227.1+ MB


去重前3,764,888条数据，去重后2,976,570条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [139]:
data_huab2017=data_hb2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [140]:
data_huab2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3393438 entries, 0 to 14709
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 258.9+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [141]:
data_huab2017_drop=data_huab2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [142]:
data_huab2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2854575 entries, 0 to 14709
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 217.8+ MB


去重前3,393,438条数据，去重后2,854,575条数据。

__两年数据合并__

In [143]:
data_huabei=pd.concat([data_huab2016_drop,data_huab2017_drop])

In [144]:
data_huabei['Station_Id_C']=data_huabei['Station_Id_C'].apply(lambda s:int(s))
data_huabei.count()

Station_Id_C    5831145
Year            5831145
Mon             5831145
Day             5831145
Hour            5831145
RHU             5831145
TEM             5831145
TEM_Max         5831145
TEM_Min         5831145
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [153]:
sta_hb_province=station[station['省份']==hb[0]]

i=1
while i < len(hb):
    sta_hb_province1=station[station['省份']==hb[i]]
    sta_hb_province=pd.concat([sta_hb_province,sta_hb_province1])
    i+=1    #提取所有华东省份的站点ID

In [154]:
sta_hb_province.count()

省份              379
Station_Id_C    379
dtype: int64

In [155]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_huabei_f=pd.merge(data_huabei,sta_hb_province,on='Station_Id_C',how='outer')
data_huabei_f['Area']='huabei'

In [156]:
data_huabei_f.count()

Station_Id_C    5831145
Year            5831145
Mon             5831145
Day             5831145
Hour            5831145
RHU             5831145
TEM             5831145
TEM_Max         5831145
TEM_Min         5831145
省份              5831145
Area            5831145
dtype: int64

华北地区站点共有379个，筛选后的数据5,831,145条。

In [186]:
data_huabei_f.to_csv('F:\\weather\\huabei.csv',index=False,encoding='utf_8_sig',sep=',')

### 华中地区

#### 数据筛选

In [63]:
sta_hz=station[station['省份']==hz[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(hz):
    sta_hz1=station[station['省份']==hz[i]]['Station_Id_C'].values.tolist()
    sta_hz.extend(sta_hz1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [64]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hz else 0 )
data_hz2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hz else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hz2016=pd.concat([data_hz2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [65]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_hz else 0 )
data_hz2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_hz else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_hz2017=pd.concat([data_hz2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [157]:
data_huaz2016=data_hz2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [158]:
data_huaz2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2497777 entries, 0 to 13437
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 190.6+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [159]:
data_huaz2016_drop=data_huaz2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [160]:
data_huaz2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068829 entries, 0 to 13437
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 157.8+ MB


去重前2,497,777条数据，去重后2,068,829条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [161]:
data_huaz2017=data_hz2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [162]:
data_huaz2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534743 entries, 0 to 13271
Data columns (total 9 columns):
Station_Id_C    1534743 non-null object
Year            1534743 non-null float64
Mon             1534743 non-null float64
Day             1534743 non-null float64
Hour            1534743 non-null float64
RHU             1534743 non-null object
TEM             1534743 non-null float64
TEM_Max         1534743 non-null float64
TEM_Min         1534743 non-null float64
dtypes: float64(7), object(2)
memory usage: 117.1+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [163]:
data_huaz2017_drop=data_huaz2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [164]:
data_huaz2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1367835 entries, 0 to 13271
Data columns (total 9 columns):
Station_Id_C    1367835 non-null object
Year            1367835 non-null float64
Mon             1367835 non-null float64
Day             1367835 non-null float64
Hour            1367835 non-null float64
RHU             1367835 non-null object
TEM             1367835 non-null float64
TEM_Max         1367835 non-null float64
TEM_Min         1367835 non-null float64
dtypes: float64(7), object(2)
memory usage: 104.4+ MB


去重前1,534,743条数据，去重后1,367,835条数据。

__两年数据合并__

In [165]:
data_huazhong=pd.concat([data_huaz2016_drop,data_huaz2017_drop])

In [166]:
data_huazhong['Station_Id_C']=data_huazhong['Station_Id_C'].apply(lambda s:int(s))
data_huazhong.count()

Station_Id_C    3436664
Year            3436664
Mon             3436664
Day             3436664
Hour            3436664
RHU             3436664
TEM             3436664
TEM_Max         3436664
TEM_Min         3436664
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [167]:
sta_hz_province=station[station['省份']==hz[0]]

i=1
while i < len(hz):
    sta_hz_province1=station[station['省份']==hz[i]]
    sta_hz_province=pd.concat([sta_hz_province,sta_hz_province1])
    i+=1    #提取所有华东省份的站点ID

In [169]:
sta_hz_province.count()

省份              272
Station_Id_C    272
dtype: int64

In [170]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_huazhong_f=pd.merge(data_huazhong,sta_hz_province,on='Station_Id_C',how='outer')
data_huazhong_f['Area']='huazhong'

In [171]:
data_huazhong_f.count()

Station_Id_C    3436664
Year            3436664
Mon             3436664
Day             3436664
Hour            3436664
RHU             3436664
TEM             3436664
TEM_Max         3436664
TEM_Min         3436664
省份              3436664
Area            3436664
dtype: int64

华中地区站点共有272个，筛选后的数据3,436,664条。

### 西南地区

#### 数据筛选

In [8]:
sta_xn=station[station['省份']==xn[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(xn):
    sta_xn1=station[station['省份']==xn[i]]['Station_Id_C'].values.tolist()
    sta_xn.extend(sta_xn1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [13]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_xn else 0 )
data_xn2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_xn else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_xn2016=pd.concat([data_xn2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [15]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_xn else 0 )
data_xn2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_xn else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_xn2017=pd.concat([data_xn2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [16]:
data_xin2016=data_xn2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [17]:
data_xin2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3532255 entries, 0 to 5543
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 269.5+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [18]:
data_xin2016_drop=data_xin2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [19]:
data_xin2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2916733 entries, 0 to 5543
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 222.5+ MB


去重前3,532,255条数据，去重后2,916,733条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [20]:
data_xin2017=data_xn2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [21]:
data_xin2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2562344 entries, 0 to 3633
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 195.5+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [22]:
data_xin2017_drop=data_xin2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [23]:
data_xin2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2220374 entries, 0 to 3633
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 169.4+ MB


去重前2,562,344条数据，去重后2,220,374条数据。

__两年数据合并__

In [24]:
data_xinan=pd.concat([data_xin2016_drop,data_xin2017_drop])

In [25]:
data_xinan['Station_Id_C']=data_xinan['Station_Id_C'].apply(lambda s:int(s))
data_xinan.count()

Station_Id_C    5137107
Year            5137107
Mon             5137107
Day             5137107
Hour            5137107
RHU             5137107
TEM             5137107
TEM_Max         5137107
TEM_Min         5137107
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [26]:
sta_xn_province=station[station['省份']==xn[0]]

i=1
while i < len(xn):
    sta_xn_province1=station[station['省份']==xn[i]]
    sta_xn_province=pd.concat([sta_xn_province,sta_xn_province1])
    i+=1    #提取所有华东省份的站点ID

In [27]:
sta_xn_province.count()

省份              387
Station_Id_C    387
dtype: int64

In [28]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_xinan_f=pd.merge(data_xinan,sta_xn_province,on='Station_Id_C',how='outer')
data_xinan_f['Area']='xinan'

In [29]:
data_xinan_f.count()

Station_Id_C    5137107
Year            5137107
Mon             5137107
Day             5137107
Hour            5137107
RHU             5137107
TEM             5137107
TEM_Max         5137107
TEM_Min         5137107
省份              5137107
Area            5137107
dtype: int64

西南地区站点共有387个，筛选后的数据5,137,107条。

In [66]:
data_xinan_f.to_csv('F:\\weather\\xinan.csv',index=False,encoding='utf_8_sig',sep=',')

### 西北地区

#### 数据筛选

In [30]:
sta_xb=station[station['省份']==xb[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(xb):
    sta_xb1=station[station['省份']==xb[i]]['Station_Id_C'].values.tolist()
    sta_xb.extend(sta_xb1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [31]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_xb else 0 )
data_xb2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_xb else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_xb2016=pd.concat([data_xb2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [32]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_xb else 0 )
data_xb2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_xb else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_xb2017=pd.concat([data_xb2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [33]:
data_xib2016=data_xb2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [34]:
data_xib2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2568063 entries, 0 to 10539
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 195.9+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [35]:
data_xib2016_drop=data_xib2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [36]:
data_xib2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2184481 entries, 0 to 10539
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 166.7+ MB


去重前2,568,063条数据，去重后2,184,481条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [37]:
data_xib2017=data_xb2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [38]:
data_xib2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1764801 entries, 0 to 10008
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 134.6+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [39]:
data_xib2017_drop=data_xib2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [40]:
data_xib2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1593979 entries, 0 to 10008
Data columns (total 9 columns):
Station_Id_C    1593979 non-null object
Year            1593979 non-null float64
Mon             1593979 non-null float64
Day             1593979 non-null float64
Hour            1593979 non-null float64
RHU             1593979 non-null object
TEM             1593979 non-null float64
TEM_Max         1593979 non-null float64
TEM_Min         1593979 non-null float64
dtypes: float64(7), object(2)
memory usage: 121.6+ MB


去重前1,764,801条数据，去重后1,593,979条数据。

__两年数据合并__

In [41]:
data_xibei=pd.concat([data_xib2016_drop,data_xib2017_drop])

In [42]:
data_xibei['Station_Id_C']=data_xibei['Station_Id_C'].apply(lambda s:int(s))
data_xibei.count()

Station_Id_C    3778460
Year            3778460
Mon             3778460
Day             3778460
Hour            3778460
RHU             3778460
TEM             3778460
TEM_Max         3778460
TEM_Min         3778460
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [43]:
sta_xb_province=station[station['省份']==xb[0]]

i=1
while i < len(xb):
    sta_xb_province1=station[station['省份']==xb[i]]
    sta_xb_province=pd.concat([sta_xb_province,sta_xb_province1])
    i+=1    #提取所有华东省份的站点ID

In [44]:
sta_xb_province.count()

省份              290
Station_Id_C    290
dtype: int64

In [45]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_xibei_f=pd.merge(data_xibei,sta_xb_province,on='Station_Id_C',how='outer')
data_xibei_f['Area']='xibei'

In [46]:
data_xibei_f.count()

Station_Id_C    3778460
Year            3778460
Mon             3778460
Day             3778460
Hour            3778460
RHU             3778460
TEM             3778460
TEM_Max         3778460
TEM_Min         3778460
省份              3778460
Area            3778460
dtype: int64

西北地区站点共有290个，筛选后的数据3,778,460条。

In [73]:
data_xibei_f.to_csv('F:\\weather\\xibei.csv',index=False,encoding='utf_8_sig',sep=',')

### 东北地区

#### 数据筛选

In [48]:
sta_db=station[station['省份']==db[0]]['Station_Id_C'].values.tolist()
i=1
while i < len(db):
    sta_db1=station[station['省份']==db[i]]['Station_Id_C'].values.tolist()
    sta_db.extend(sta_db1)
    i+=1    #提取所有华东省份的站点ID

__2016年__

In [49]:
data_m=pd.read_table(path_2016+file_2016[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_db else 0 )
data_db2016=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2016)-1:
    data_set=pd.read_table(path_2016+file_2016[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_db else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_db2016=pd.concat([data_db2016,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


__2017年__

In [50]:
data_m=pd.read_table(path_2017+file_2017[0],delim_whitespace=True)#多个空值作分隔符,读取第一个文件
data_m['Station_Id_C']=data_m['Station_Id_C'].apply(lambda s: s if s in sta_db else 0 )
data_db2017=data_m[data_m['Station_Id_C']>0]

i=1
while i<=len(file_2017)-1:
    data_set=pd.read_table(path_2017+file_2017[i],delim_whitespace=True)
    data_set['Station_Id_C']=data_set['Station_Id_C'].apply(lambda s: s if s in sta_db else 0 )
    data_sets=data_set[data_set['Station_Id_C']>0]
    data_db2017=pd.concat([data_db2017,data_sets])
    i+=1
else:
    print('merge complete!')

merge complete!


#### 数据质控

__2016年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [51]:
data_dongb2016=data_db2016[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [52]:
data_dongb2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1812547 entries, 0 to 13116
Data columns (total 9 columns):
Station_Id_C    object
Year            float64
Mon             float64
Day             float64
Hour            float64
RHU             object
TEM             float64
TEM_Max         float64
TEM_Min         float64
dtypes: float64(7), object(2)
memory usage: 138.3+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [53]:
data_dongb2016_drop=data_dongb2016.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [54]:
data_dongb2016_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503661 entries, 0 to 13116
Data columns (total 9 columns):
Station_Id_C    1503661 non-null object
Year            1503661 non-null float64
Mon             1503661 non-null float64
Day             1503661 non-null float64
Hour            1503661 non-null float64
RHU             1503661 non-null object
TEM             1503661 non-null float64
TEM_Max         1503661 non-null float64
TEM_Min         1503661 non-null float64
dtypes: float64(7), object(2)
memory usage: 114.7+ MB


去重前1,812,547条数据，去重后1,503,661条数据。

__2017年__

__保留关键指标：相对湿度（0~100），平均、最大、最小温度（-50~+100），风速（RHU、TEM、TEM_Max、TEM_Min、WIN_D_Avg_2mi）__

In [55]:
data_dongb2017=data_db2017[['Station_Id_C','Year','Mon','Day','Hour','RHU','TEM','TEM_Max','TEM_Min']]

In [56]:
data_dongb2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1603234 entries, 0 to 12072
Data columns (total 9 columns):
Station_Id_C    1603234 non-null object
Year            1603234 non-null float64
Mon             1603234 non-null float64
Day             1603234 non-null float64
Hour            1603234 non-null float64
RHU             1603234 non-null object
TEM             1603234 non-null float64
TEM_Max         1603234 non-null float64
TEM_Min         1603234 non-null float64
dtypes: float64(7), object(2)
memory usage: 122.3+ MB


__删除重复值：每次导出数据可能有重叠，合并“测定站-年-月-日-时间”删重__

In [57]:
data_dongb2017_drop=data_dongb2017.drop_duplicates(['Station_Id_C','Year','Mon','Day','Hour'],keep='first')

In [58]:
data_dongb2017_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385171 entries, 0 to 12072
Data columns (total 9 columns):
Station_Id_C    1385171 non-null object
Year            1385171 non-null float64
Mon             1385171 non-null float64
Day             1385171 non-null float64
Hour            1385171 non-null float64
RHU             1385171 non-null object
TEM             1385171 non-null float64
TEM_Max         1385171 non-null float64
TEM_Min         1385171 non-null float64
dtypes: float64(7), object(2)
memory usage: 105.7+ MB


去重前1,603,234条数据，去重后1,385,171条数据。

__两年数据合并__

In [59]:
data_dongbei=pd.concat([data_dongb2016_drop,data_dongb2017_drop])

In [60]:
data_dongbei['Station_Id_C']=data_dongbei['Station_Id_C'].apply(lambda s:int(s))
data_dongbei.count()

Station_Id_C    2888832
Year            2888832
Mon             2888832
Day             2888832
Hour            2888832
RHU             2888832
TEM             2888832
TEM_Max         2888832
TEM_Min         2888832
dtype: int64

__为方便后续数据整理，给每个地区增加2个数据标签：1.自然地理分区，2.省份自治区__

In [61]:
sta_db_province=station[station['省份']==db[0]]

i=1
while i < len(db):
    sta_db_province1=station[station['省份']==db[i]]
    sta_db_province=pd.concat([sta_db_province,sta_db_province1])
    i+=1    #提取所有华东省份的站点ID

In [62]:
sta_db_province.count()

省份              186
Station_Id_C    186
dtype: int64

In [63]:
#使用station数据集先标明各个站点的省份自治区，然后根据area数据集标明地理分区
data_dongbei_f=pd.merge(data_dongbei,sta_db_province,on='Station_Id_C',how='outer')
data_dongbei_f['Area']='dongbei'

In [64]:
data_dongbei_f.count()

Station_Id_C    2888832
Year            2888832
Mon             2888832
Day             2888832
Hour            2888832
RHU             2888832
TEM             2888832
TEM_Max         2888832
TEM_Min         2888832
省份              2888832
Area            2888832
dtype: int64

华中地区站点共有186个，筛选后的数据2,888,832条。

In [65]:
data_dongbei_f.to_csv('F:\\weather\\dongbei.csv',index=False,encoding='utf_8_sig',sep=',')